In [49]:
from ColonyMgmt import * 
import seaborn as sns

# wish list

* highlight file time if > 1 hr old
* google scripts API to circumvent this issue entirely
* clean up wrapping and helper functions

## some useful helper functions to consider
Convert a given string to {current-date}{string}.csv

```
makefn('string')
```

### helpful consolidators
Creates two dataframes of all cages + dispensible % > .2, and either saves them independently or pops them back for  further manip.
```
makeConsolidations(df, save = True, popBack = False)
```    
                  

### helpful for euthanizing many animals
```
df.fillna("-").groupby(['Cage','DOD']).agg({'Tag':'unique', 'Lineage': 'unique'}).to_csv(fn)
```

In [29]:
file = '05227_mastersheet - HopkinsColony.tsv'
cats = ['Tag', 'Ear', 'Sex', 'Color', 'DOB', 'Lineage', 'Genotype', 'Father', 'Mother']

In [3]:
ts = os.path.getmtime(file)
print("mtime:", dt.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

mtime: 2020-12-15 18:39:09


In [22]:
df = importCleanDF(file,
                   dropDeads=False,
                   dropTransfers=True)

df = df[~df.DOD.str.contains("euthan|fd")]

Read in CSV: 05227_mastersheet - HopkinsColony.tsv.
File was last modified at: 2020-12-15 18:41:17

Animals without gender given temp. gender
Animals without cage given temp. cage
Animals without specified lineage given NA
calculted total dispensible counts
calculated total cage sizes
calculated % disp
certainly cage reorg time
after removing transferred {'numCages': 199, 'numMice': 513}
Parsed ear tags for Excel formatting


In [23]:
# check that lineages are correct
for num, el in enumerate(df.Lineage.unique()):
    print(num, el)

0 1763PB#7
1 LSL-Cas9 x SynCre
2 1763PB#3
3 LSL-Cas9 x hCD2
4 1853PBF#1
5 1685ins21#2
6 Cas9
7 Cas9 x PB3
8 PB#21


In [48]:
# mask = ~df.TagClean.isin(tier1) 
experimental = df.Lineage.str.contains("Cas9 x")
PB = df.Lineage.str.contains("PB#")
tier1 = df.DOD.str.contains("tier1_confirmed|breed")
femalesM = (df.Sex.str.lower() == "f")
malesM = (df.Sex.str.lower() == "m")
not_ancient = df.DOB > '2020-04-01'
ins = df.Lineage.str.contains("ins")
not_new = df.DOB < '2020-10-20'
pb3 = df.Lineage.str.contains("PB#")


In [23]:
df[ins & not_new][cats+['DOD']].to_clipboard(index=False)


In [17]:
# a, b = makeConsolidations(df, save = True, popBack = True)

Saved CSV with fns AllCages_2020-December-01.csv
Saved CSV with fns FemaleCons_2020-December-01.csv
returning 2 frames


In [5]:
df.groupby(['Lineage', 'DOD']).agg({'Cage': 'nunique', 'Sex':'unique', 'Tag': ['nunique','unique'], })

Cage        Sex  \
                                                      nunique     unique   
Lineage           DOD                                                      
1685ins21#2       -                                        12     [M, F]   
                  breeding-confirmed-201014                 4     [M, F]   
                  dispensible                               2     [M, F]   
                  fertile male                              1        [M]   
1763PB#3          -                                         2        [M]   
                  dispensible                               1        [M]   
                  experimentalbreeder-201029                2        [M]   
                  pb21breeder_tier2-201015                  3        [F]   
                  pb21breeders_tier1_confirmed-201015       3        [F]   
                  pb21breeders_tier2_201015                 1        [F]   
                  pb3breeder_tier1_confirmed-201101         7     [F, M]   
                  pb3breeder_tier2_putative-201104          1        [M]   
1763PB#7          -                                        17     [F, M]   
                  breeder-may20                             3        [F]   
                  breeding-confirmed-201019                 5  [F, m, M]   
                  breeding-confirmed-201021                 1        [F]   
                  dispensible                               1        [M]   
                  fertile male                              4        [M]   
                  pb21breeder_tier2-201015                  1        [M]   
                  pb21breeders_tier1_confirmed-201015       3     [M, m]   
1853PBF#1         -                                         2        [M]   
                  breeder-may20                             1        [F]   
                  dispensible                               2     [M, F]   
                  pbfbreeder_tier1_confirmed-201101         3     [M, F]   
                  pbfbreeder_tier2_putative-201104          4     [M, F]   
Cas9              -                                         9     [M, F]   
                  experimental breeder                      2        [F]   
Cas9 x PB3        -                                         2     [M, F]   
                  dispensible                               1        [M]   
                  vickie (experimental)                     2     [F, M]   
LSL-Cas9 x SynCre -                                         1        [M]   
                  breeder-jun01                             1        [M]   
                  breeder-jun01backup                       1        [M]   
                  breeding                                  1        [F]   
                  dispensible                               4     [F, M]   
LSL-Cas9 x hCD2   -                                         5     [M, F]   
                  breeder-jun01                             1        [M]   
                  breeder-jun01backup                       1        [F]   
                  breeding                                  2     [M, F]   
                  dispensible                               4     [M, F]   

                                                          Tag  \
                                                      nunique   
Lineage           DOD                                           
1685ins21#2       -                                        39   
                  breeding-confirmed-201014                11   
                  dispensible                               2   
                  fertile male                              1   
1763PB#3          -                                         5   
                  dispensible                               1   
                  experimentalbreeder-201029                2   
                  pb21breeder_tier2-201015                  3   
                  pb21breeders_tier1_confirmed-201015       6   
        

# speciality functions - check breedings, splitting genotypes

In [22]:
source_dir = "Breedings"
date = "2020-10-09"

breeders = getBreedingList(source_dir, date)
tier2 = set(breeders.query('SourceFn.str.contains("compositions2")')['Tag'])
tier1 = set(breeders.query('SourceFn.str.contains("compositions1")')['Tag'])

# syntax to check if breeder list against colony list
df[df['Tag'].isin(tier1) & df['Tag'].notnull()] 

#if any of the breeding pairs have an NA in them -- i.e. a pair that is set up with only one female, this will match *all* animals that also have a null tag. Check with the below line to see if this is the case.
breeders.query('Tag.isnull()')


,Tag,Lineage,Date,SourceFn
79,NaN,PBF,2020-10-09,Breedings/PBF-2020-10-09_cage_compositions2(se...


In [27]:
breeders[breeders.Tag.isin(tier2)].sort_values('Tag')

,Tag,Lineage,Date,SourceFn
37,5072,PB7,2020-10-09,Breedings/PB7-2020-10-09_cage_compositions2(se...
6,5134,PB7,2020-10-09,Breedings/PB7-2020-10-09_cage_compositions2(se...
13,5134,PB21,2020-10-0,Breedings/PB21-2020-10-09_cage_compositions1(b...
7,5135,PB7,2020-10-09,Breedings/PB7-2020-10-09_cage_compositions2(se...
26,5136,PB21,2020-10-0,Breedings/PB21-2020-10-09_cage_compositions2(s...
17,5187,PBF,2020-10-09,Breedings/PBF-2020-10-09_cage_compositions2(se...
38,5197,PB7,2020-10-09,Breedings/PB7-2020-10-09_cage_compositions2(se...
68,5280,PB7,2020-10-09,Breedings/PB7-2020-10-09_cage_compositions2(se...
69,5283,PB7,2020-10-09,Breedings/PB7-2020-10-09_cage_compositions2(se...
16,5347,PBF,2020-10-09,Breedings/PBF-2020-10-09_cage_compositions2(se...


In [13]:
# df[df['Tag'].isin(tier2) & df['Tag'].notnull()] 

In [20]:
## TEMPORARY CODE
transfers = df.query('DOD.str.contains("glatzer|west")')
transfers.groupby(['DOD']).agg({'Cage': ['nunique','unique'], 'Tag': ['nunique','unique'], 'Lineage':'unique'}).to_clipboard()

In [33]:
# to split genotyping info
df = splitGenotypes(df)

In [ ]:
df.groupby(['DOD']).agg({'Cage': 'nunique', 'Tag': ['nunique','unique'], 'Lineage':'unique'})

In [ ]:
mask = ~df.TagClean.isin(tier1) 
experimental = df.Lineage.str.contains("Cas9 x")
PB = df.Lineage.str.contains("PB#")
tier1 = df.DOD.str.contains("tier1_confirmed|breed")
femalesM = (df.Sex.str.lower() == "f")
malesM = (df.Sex.str.lower() == "m")
not_ancient = df.DOB > '2020-04-01'


In [ ]:

df['mutationScore'] = df['Genotype'].str.split(":", expand = True)[1].astype(float)
df['early_and_mid_ct'] = df['E_num'] + df ['I_num']

In [ ]:
df[mask & PB & not_ancient & ~tier1 & malesM][['Tag', 'Cage', 'Ear', 'Sex', 'Color', 'DOB', 'DOD', 'Lineage', 'early_and_mid_ct','summedhgRNAs_active']].sort_values('early_and_mid_ct', ascending = False).to_clipboard(index = False)

In [ ]:
df[mask & experimental & not_ancient & ~tier1][['Tag', 'Cage', 'Ear', 'Sex', 'Color', 'Name', 'Genotype', 'DOB', 'DOD', 'Lineage', 'E_num', 'I_num', 'S_num',
       'ina_num', 'summedhgRNAs', 'summedhgRNAs_active']].sort_values([['E_num', 'I_num'])]
# .to_clipboard(index = False)

In [ ]:
df[mask & experimental & not_ancient & ~tier1][['Tag', 'Cage', 'Ear', 'Sex', 'Color', 'Name', 'Genotype', 'DOB', 'DOD', 'mutationScore']].sort_values('mutationScore', ascending = False).to_clipboard(index = False)

In [ ]:
df.query("Lineage.str.contains('PB')").groupby(["Lineage", 'DOD']).agg({'Tag':'nunique'})

In [ ]:
breedersM = df['Tag'].isin(set(breeders['Tag']))

euthReqM = (df['DOD'] == 'requested euthanasia')
fdM = df.DOD.str.contains("fd")
experimentalM = df.DOD.str.contains("experimental|~breeder") | df.Lineage.str.contains("Exp")
pbM = df.Lineage.str.contains("PB#")
insM = df.Lineage.str.contains("ins")





In [ ]:
working = df[ ~euthReqM & ~fdM].copy().reset_index()

In [ ]:
working['mutationScore'] = working['Genotype'].str.split(":", expand = True)[1].astype(float)

In [ ]:
working[easyCats + ['DOD','Genotype', 'mutationScore', 'Father', 'Mother']].sort_values(['Sex', 'mutationScore'], ascending = False)
# .to_clipboard(index = False)

In [ ]:
working.query("DOD.str.contains('putative')")[easyCats +['DOD']]

In [ ]:
working

In [ ]:
df[malesM & pbM & ~euthReqM][easyCats + ['DOD', 'E_num', 'I_num', 'S_num', 'summedhgRNAs', 'summedhgRNAs_active']].sort_values('summedhgRNAs_active', ascending = False).query('~DOD.str.contains("confirmed|xcas9|transferred|tier1|breeder-may20")')

In [ ]:
meese = df.query(
    'DOD.str.contains("pbf|pb3") & DOD.str.contains("tier1")').sort_values(
        ['Sex', 'Lineage'], ascending=False)
meese.groupby("Cage", sort = False).agg({
    'Tag': 'unique', 'CageSize':'mean',
    'Ear': 'unique',
    'Color': 'unique',
    'Lineage':'unique', 'Sex': 'unique'
})

In [ ]:
meese[easyCats].to_clipboard(index = False)

In [ ]:
# cages_find.head()

In [ ]:
# cages_find.groupby("Cage").agg({'Tag': ['count', 'unique'], 'Ear': 'unique', 'Color': 'unique'}).to_clipboard(sep = "\t")

In [ ]:
# df.query('DOD == "breeding-putative-201015"')[easyCats].sort_values('DOB').to_csv(makefn('PB7_breeders'), sep = "\t")



In [ ]:
# ## vickie cages
# # mutant:
# AA0105 AA0105
# AA0056 5171-5174 (WT / syn cre relevant)

In [ ]:
# df.query('Cage.str.contains("AA0105|AA0056")')[easyCats].to_clipboard(index = False, sep = "\t")

In [ ]:
# df.query('Cage.str.contains("AA0105|AA0056")')[['Tag', 'Cage', 'Ear', 'Sex', 'Color', 'DOB', 'Lineage']].to_clipboard(index = False, sep = "\t")

In [ ]:
# mice = df.fillna("-").query('Tag.str.contains("5049|5253|5254|5242")')
# cages = mice.Cage.unique()

In [ ]:
# # cages = mice.Cage.unique()
# df[df.Cage.isin(cages)][easyCats].to_clipboard(index = False, sep = "\t")

In [ ]:
# df[df.Cage.isin(cages)][['Tag', 'Cage', 'Ear', 'Sex', 'Color', 'DOB', 'Lineage']]

# code from summing total hgRNA counts for collecting animal samples

## splitting hgRNA genotypes into columns

In [ ]:
## speciality speed code for re-splitting genotypes by hgRNA tyep
def splitGenotypes(df):
    '''for splitting genotypes into hgRNA categories and summing to pick breeders when Reza hasn't, or for distribution, or whatever'''
    split = df['Genotype'].str.split(r'(\w,)', expand=True)
    split[6] = split[6].str.extract('(\d+)')
    split = split.rename({
        0: "E",
        2: 'I',
        4: "S",
        6: 'ina'
    }, axis=1).drop(columns=[1, 3, 5])

    newcols = []
    oldcols = []
    for col in split:
        colN = str(col) + "_num"
        split[colN] = pd.to_numeric(split[col], errors='coerce')
        oldcols.append(col)
        newcols.append(colN)

    split['summedhgRNAs'] = split[newcols].sum(axis=1)
    split['summedhgRNAs_active'] = split[newcols[0:3]].sum(axis=1)
    split = split.drop(oldcols[0:4], axis=1)
    df = df.merge(split, left_index=True, right_index=True)
    return df

In [ ]:
df = splitGenotypes(df)

In [ ]:
breedersMask = df['Tag'].isin(set(breeders['Tag']))
euthanasiaMask = (df['DOD'] == 'requested euthanasia')
fdMask = df.DOD.str.contains("fd|breeder-may|experimental")
pbMask = df.Lineage.str.contains("PB#")
fMask = (df.Sex.str.lower() == "f")

In [ ]:
df[~breedersMask & ~euthanasiaMask & ~fdMask & pbMask & ~fMask].sort_values(['E_num', 'I_num', 'S_num'])[easyCats]

In [ ]:
# .to_clipboard(index=False, sep = "\t")
df.query('DOD.str.contains("zack")')[['Tag','Genotype', 'summedhgRNAs', 'summedhgRNAs_active']].to_csv('ZackTransferGenotypes.tsv', sep = '\t', index=False) 

## query each lineage + create CSV from it

# checking breedings against spreadsheet

In [ ]:
source_dir = "Breedings"
date = "2020-10-09"
fns = glob.glob(f"{source_dir}/*{date}_cage*")
data = []

for fn in fns:
    frame = pd.read_csv(fn, sep="\t")
    frame['Lineage'], frame['Date'] = fn[10:24].split("-", 1)
    data.append(frame)

animals = pd.concat(
    data, ignore_index=True)  #dont want pandas to try an align row indexes

animals = animals.applymap(lambda x: x.strip("t") if isinstance(
    x, str) else x)  #remove t for easier comparison to big frame applys

tagCols = ['Male', 'Female1',
           'Female2']  #grab columns with animals for flattening

data = []
for col in tagCols:
    frame = pd.concat((animals[i] for i in [col, 'Lineage', 'Date']),
                      axis=1).rename({col: 'Tag'}, axis=1)
    data.append(frame)

#            #, animals['Lineage'], animals['Date']]
# #     data.append(frame)
breeders = pd.concat(
    data, ignore_index=True)  #dont want pandas to try an align row indexes;

## check that all animals are correctly marked in the spreadsheet

In [ ]:
## #### parsing breedings etc - check that all breeders are marked as such on the sheet
breedrs = pd.merge(df, breeders, left_on ='TagClean', right_on = 'Tag', how = 'right')
if len(breedrs.DOD.unique()) > 1:
    print("Not all breeders marked")
else:
    print("Only one DOD found, {}".format(breedrs.DOD.unique()[0]))
    
list(set(breeders['Tag'])-set(breedrs['TagClean']))

In [ ]:
breedrs.query('DOD.str.contains("pbfbreeder_tier1", na = False)').groupby("Cage").agg({'Cage': 'nunique', 'TagClean': ['nunique','unique'], 'Lineage_x':'unique'})

# something eles happens here? something about querying for samples of a specific type

In [ ]:
# def query(df, lineage, disp=1):
#     result = df.query(
#         'Lineage.str.contains("{}")'.format(lineage))
#     if disp:
#         result = result.query('DispPercent >= {}'.format(disp))
#     result = result.sort_values("summedhgRNAs_active", ascending=False)
#     return result

# frames = {}
# lineages = ['#3', '#7', 'ins21', '-Cas9', 'PBF']

# for line in lineages:
#     frame = query(df, line).head(5)
#     frames.update({line: frame})
# for frame in frames:
#     fn = makefn(frame)
#     frames[frame].to_csv(fn, sep="\t")
#     df[['fCas', 'transgene']] = df['Genotype'].str.split(",", 1, expand = True)
# frame = query(df, 'fCas9').query('transgene.str.contains("-")')
# frame = frame.sort_values(['fCas', 'transgene'], ascending = [True, True]).head(10)
# fn = makefn('creFlipped-negative_ampliconSamples')
# frame.to_csv(fn, sep ='\t')


In [ ]:
##############################################

# assessing parental genotypes to consider offspring (for fCas9 maintenance/breeding)

In [ ]:
df.Lineage = df.Lineage.fillna("--")
fathers = df[df.Lineage.str.contains('fCas9')].Father.unique().flatten()
mothers = df[df.Lineage.str.contains('fCas9')].Mother.unique().flatten()
rents = np.append(fathers, mothers)
parentList = []
for i in rents:
    parentList.extend(i.split("/")) 
parents = df.loc[df['Tag'].isin(parentList)]

In [ ]:
## easy categories
plateCats =  ['Genotype','PlateID', 'Row', 'Column',
       'Position']
cats = easyCats + plateCats

In [ ]:
df[['fCas', 'transgene']] = df['Genotype'].str.split(",", 1, expand = True)

In [ ]:
df.Lineage = df.Lineage.fillna("--")
linMask = df.Lineage.str.contains('fCas9')
genoMask = df.Genotype.isnull()
fCas9 = df[linMask].fillna("-")
H_linMask = fCas9.Lineage.str.contains('hCD2')
S_linMask = fCas9.Lineage.str.lower().str.contains('syn')

In [ ]:
df.query("Tag == '5326'")

In [ ]:
fCas9.groupby(['Sex', 'DOD', 'fCas', 'transgene']).agg({'Genotype': 'count',
                                                   'Tag':'unique', 'Cage':'unique', 
                                                  'DOB': ['min', 'max']})

In [ ]:
cages = df.groupby(["Cage", "Sex"]).agg({'Color':'unique',
                                         'DOD':'unique',
                                         'Lineage':'unique'})   #yay!

In [ ]:
df.groupby('Cage').agg({
    'dispensible': 'mean',
    'Cage': 'size',
    'Sex': 'unique',
    'DOD': 'unique',
    'Tag': 'unique',
    'Color': 'unique',
    'Ear': 'unique',
    'Lineage': 'unique'
}).sort_values('dispensible', ascending = False)

In [ ]:
df[df.Sex == "F"].groupby(['Cage']).agg({'Cage': 'size',
                                         'DOD': 'unique',
                                         'Tag':'unique',
                                        'Color' : 'unique',
                                         'Ear' : 'unique',}).sort_values

In [ ]:
df[df.Sex == "F"].groupby(['Cage']).agg({'Cage':'size',
                                         'Color':'unique',
                                         'DOD':'unique',
                                         'Lineage':'unique'})   #yay!

# retrieve animals with null genotypes in a certain lineage + get their well plates

In [ ]:
insGeno = df[(df.Lineage.str.contains("ins21")) & (df.Genotype.isnull())]

In [ ]:
locs = pd.read_pickle('96-wellLocs.pkl').sort_values(
        ['Column',
         'Row']).reset_index(drop=True)  # impt standard plate indexing
# locs.join(df, rsuffix = "_init")
# insGeno.sort_values(join(locs, rsuffix = 'init')
locs.join(insGeno, rsuffix='init')

In [ ]:
insGeno.pivot(index='WellRow', columns='WellCol',
                      values=['Tag', 'Plate'])

# generate a general colony pivot table by lineage, sex, and DOB with total numbers

In [ ]:
saveFn = makefn(file.replace(" ", "")+'_summary')

remapDict= {'ins': '1685ins21#2',
            'pb3' : '1763PB#3',
            'pb7' : '1763PB#7',
            'pbf': '1853PBF#1'}
for key, value in remapDict.items():
    df.Lineage = df.Lineage.apply(lambda row: value if key in row.lower() else row)
grouped = df.groupby(['Lineage',
            'Sex',
            pd.Grouper(key='DOB',freq='q')]).size().unstack('DOB').reset_index()
grouped['Total'] = grouped.sum(axis=1)
grouped.fillna("-").to_csv(saveFn)

In [ ]:
saveFn = makefn(file.replace(" ", "")+'_femaleConsolidation')

In [ ]:
df.groupby(["Cage"]).agg({'Cage': 'count',  'Sex': 'unique',
                                 'Color':'unique',
                                'DOD': 'unique'})

In [ ]:
mating = tagSearch(ins + PB3 + PB7 + PBF + cre)
df[mating].sort_values('Lineage').to_csv('file.csv')

# break - 2019 and previous code snippets

In [ ]:
CreMask = (df.Lineage.str.contains("fCas", na=False))
PBFMask = (df.Lineage.str.contains("PBF", na=False))
insMask = tagSearch("t2620 t2632 t2623 t2463 t2625 t2622 t2616 t2626 t2633 t2720 t2721 t2722 t2755 t2750 t2751 t2415 t2445")
alreadyShipped = (df.Cage.str.contains("HOP", na= True))
traveling = (df.DOD.str.endswith("hopkins| ", na = False))

In [ ]:
shipped_2 = df[(df.DOD.str.endswith("hopkins", na = False))]

In [ ]:
shipped_2.sort_values(["Cage", "Tag"])[['Cage', 'Sex', 'Tag', 'Ear', 'Color', 'DOB']].to_clipboard(excel=True, index=False)

In [ ]:
shipped_2.groupby(["Cage"]).agg({'Cage': 'count', 'Sex':'unique'}).to_clipboard(excel=True)

In [ ]:
hop = df[((traveling & (CreMask | PBFMask)) | insMask ) & ~alreadyShipped]

In [ ]:
hop.groupby(["Cage", "Sex"]).agg({'Cage': 'count'})

In [ ]:
hop.groupby(["Sex", "Cage", "Tag"]).agg({'Lineage': 'unique',
                                         'Color': 'unique',
                                        'Ear': 'unique'})

In [ ]:
hop.groupby(['Sex','Cage']).agg({'Cage':'count'})

In [ ]:
hop.sort_values(['Sex', "Lineage", 'Tag'])

In [ ]:
df[(df.Genotype.isnull()) \
   & (df['DOD'] != "Dispensible")].fillna("x").groupby(["Lineage", 'Cage']).agg({'Sex': 'count',
                                                                                   'Tag':'unique',
                                                                                   'Name': 'unique' })

In [ ]:
#mask for sorting centrifuge tube samples
freezerMask = (df.Tag.notnull()) & \
    (~df['Notebook info'].str.startswith("T|Plate", na=False))&\
     (df.Lineage != 'J024858-Cas9')

#and the results
df[freezerMask].sort_values("Tag")

In [ ]:
earlyMask, lateMask, fMask = collectAgeRange(df, '2019-12-01',
                                             28, 0 , "", "F")
earlyMask, lateMask, mMask = collectAgeRange(df, '2019-12-01',
                                             40, 0 , "", "M")

inc = df[mMask].append(df[fMask]) #combine

inc['DOD'], inc['parsedDOD'] =  zip(*inc.DOD.apply(lambda row: cleanDOD(row))) #parse DOD
inc = inc[(~inc.parsedDOD.notnull()) \
          & (~inc.DOD.str.contains('sperm|Diss|f.d.',
                                   na = False))] #drop assorted dead mice

In [ ]:
inc.fillna('n/a').groupby(['Lineage',
                           'DOD',
                           'Sex']).agg({'DOB': ['count',
                                                'min',
                                                'max']})

In [ ]:
inc[inc.DOD.str.contains('Hopkins', na= False)].groupby("Sex").agg({'Sex': 'count',
                                                    'Cage': 'nunique'})


In [ ]:
noMice, noCages

In [ ]:
df[df.Plate == "T885765"].sort_values(['WellRow',
                                       'WellCol'])[['Position',
                                                    'Tag',
                                                    'Lineage']].to_clipboard(excel=True, header = False, index = False)

In [ ]:
def is_coherent(seq):
    return seq == range(seq[0], seq[-1]+1)

In [ ]:
MMRRC_cats = ['Tag',
           'Cage',
           'Ear',
           'Sex',
           'Color',
           'Genotype',
           'DOB',
           'Lineage']

In [ ]:
# earlyMask, lateMask, mMask = collectAgeRange(df,
#                                              '2019-09-16',
#                                              13, 8,
#                                              "PB#", "M")
# earlyMask, lateMask, fMask = collectAgeRange(df, 
#                                              '2019-09-16',
#                                              4, 1,
#                                              "PB#", "F")
# shippedMask = (df.DOD.str.contains("mrrc|opkins",
#                                    na=False)) & (df.Lineage.str.contains("PB#")) #PBs marked for any maintenance movement

# df[mMask|fMask|shippedMask].groupby(["Lineage",
#                                      "Sex",
#                                      "DOD"]).agg({'Tag':'count',
#                                                    'DOB': ['first', 'last']}) #

dispense, cSorted = makeOutputs(df)
cSorted.to_clipboard(excel=True, index = True)

In [ ]:
earlyMask, lateMask, incMask = collectAgeRange(df, '2019-09-16', 4, 1, "", "F")
df[incMask][['Tag',
             'Cage',
             'Ear',
             'Sex',
             'Color',
             'DOB',
             'Genotype',
             'Lineage']].groupby("Lineage").count()

In [ ]:
earlyMask, lateMask, incMask = collectAgeRange(df, '2019-10-04', 10, 0 , "", "")
#inc = df[incMask]
#inc = inc.fillna("BLANK")

# [['Tag',
#              'Cage',
#              'Ear',
#              'Sex',
#              'Color',
#              'DOB',
#              'Genotype',
#              'Lineage']]

In [ ]:
# df[df.DOD == 'Shipped to mmrrc on 9/10'][['Tag',
#              'Color',
#              'Ear',
#              'Cage', 
#              'Sex',
#              'Genotype',
#              'DOB',
#              'Lineage']].sort_values(by = ['Cage',
#                                            'Tag'])\
#                 .fillna("missing").to_clipboard(index = False,
#                                                excel = True)

In [ ]:
# ## MICE WITH UNKNOWN FATE
# df[(df.DOD.isnull())].groupby(["Lineage", "Cage"]).agg({'DOB': ['max', 
#                                                              'min', 
#                                                              'count'],
#                                                         'Sex': 'max',
#                                                         'Name': 'unique'}).fillna("")

In [ ]:
########## selecting mice by age#########
shipDate = '2019-09-16'
early1, late1, mask1 = collectAgeRange(df, shipDate, 13, 8, "PB#", "M")
early2, late2, mask2 = collectAgeRange(df, shipDate, 9, 6.5, "PB#", "F")

males_20190916  = df[mask1].sort_values(["Lineage", "Genotype"])
females_20190916 = df[mask2].sort_values(["Lineage", "Genotype"])

shipment_20190916 = males_20190916.append(females_20190916)

# Sep09_shipment.sort_values(["Lineage",
#                             "Sex",
#                             "Cage"]).to_clipboard(excel=True,
#                                                   index = False)

# Sep09_shipment.groupby(["Lineage",
#                         "Sex",
#                         "Cage"])['Tag',
#                                  'Genotype'].count().to_clipboard()

In [ ]:
plate_reseq.sort_values(["Tag"])[['Tag',
                                  'Lineage',
                                  'Plate',
                                  'WellRow',
                                  'WellCol']].to_clipboard(excel = True, index= False)

In [ ]:
df['Tag'] = 't' + df['Tag'].astype(str)

In [ ]:
KL14_plateB = df[df['Plate'].str\
   .contains("T885773|T885761", na= False)]\
    .sort_values(by=['Plate', 'WellRow', 'WellCol']).reset_index(drop = True)

In [ ]:
KL14_plateB[['Tag','Lineage','Plate', 'WellRow', 'WellCol']]\
    .to_clipboard(excel=True, index=False)

In [ ]:
KL14_plateA = df[df['Plate'].str\
   .contains("T885772", na= False)]\
    .sort_values(by=['Plate', 'WellRow', 'WellCol']).reset_index(drop = True)

In [ ]:
df[df['Plate'].str\
   .contains("T88572|T885761|T885772|T885773",
             na = False)][['Tag', 'Plate', 'Well']]\
    .sort_values(by=['Plate', 'Well'])\
    .to_clipboard(excel=True, index= False)

In [ ]:
KL14_plateA[['Tag','Lineage','Plate', 'WellRow', 'WellCol']]\
    .to_clipboard(excel=True, index=False)

In [ ]:
#KL14_plateA.reindex('WellCol', method='ffill')

In [ ]:
KL14_plateA.groupby('WellRow').Lineage.count()

In [ ]:
KL14_plateA

In [ ]:
KL14_plateA[['Tag', 'Plate', 'WellRow', 'WellCol']]\
    .sort_values(by=['Plate', 'WellRow', 'WellCol'])\
    .to_clipboard(excel=True, index= False)

In [ ]:
## lineage stats over time
df.groupby([pd.Grouper(key="DOB", freq = "M"),
            "Lineage"]).Cage.count().unstack('DOB')

In [ ]:
def dfContains(string, column = "Cage", keep_empties = False, case = False, df = df):
    '''given a cage number/s (formatted as "nrbxxxx|nrbxxx") etc, the indexes of entries where true'''
    return df[column].str.contains(string, na = keep_empties, case = case)

In [ ]:
df[df.DOD.str.contains("Sperm", na = False)].groupby('Lineage').count()

In [ ]:
df[(dfContains("1036|1037"))
   & (df.Sex == "M")][['Tag',
                      'Cage',
                       'Ear',
                       'Sex',
                       'DOB',
                       'Notes']]

In [ ]:
PB = ['1763PB#7', '1763PB#3']
counted = df.groupby("Cage").size().reset_index(name = 'Total')
df = df.merge(counted)
PB_solo_males = df[(df.Lineage.isin(PB))&(df.Sex == "M")&(df.Total == 1)]